In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(330)
cluster

from dask.distributed import Client
client = Client(cluster)
client



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://172.30.100.1:46174 Dashboard: http://172.30.100.1:44972/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [2]:
!squeue -u albert7a

import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 100:
        break
    time.sleep(1)
print(nb_workers)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8194693     hsw24 make_pro albert7a PD       0:00      1 (None)
           8194694     hsw24 make_pro albert7a PD       0:00      1 (None)
           8194695     hsw24 make_pro albert7a PD       0:00      1 (None)
           8194696     hsw24 make_pro albert7a PD       0:00      1 (None)
           8194697     hsw24 make_pro albert7a PD       0:00      1 (None)
           8194698     hsw24 make_pro albert7a PD       0:00      1 (None)
           8194689     hsw24 make_pro albert7a  R       0:01      1 n2096
           8194690     hsw24 make_pro albert7a  R       0:01      1 n2331
           8194691     hsw24 make_pro albert7a  R       0:01      1 n2546
           8194692     hsw24 make_pro albert7a  R       0:01      1 n2547
75


In [3]:
import warnings
warnings.filterwarnings("ignore")
import dask 
import numpy as np
import xarray as xr
import time
import os 
import time 
import glob
import numcodecs

import zarr

%matplotlib inline


In [4]:
files=sorted(glob.glob('/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002*-S/*/eNATL60*gridT-2D*.nc'))
for file in files:
    print(file)

/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090630-20090630.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090701-20090701.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090702-20090702.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090703-20090703.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090704-20090704.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00399601-00410400/eNATL60-BLB002_1h_20090705_20090709_gridT-2D_20090705-20090705.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00399601-00410400/eNATL60-BLB002_1h_20090705_20090709_gridT-2D_20090706-20090706.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL

In [5]:
# these are variables I just want to drop forever

drop_vars = ['nav_lat', 'nav_lon', 'somxl010', 'sosaline', 'sosstsst']
extra_coord_vars = ['time_counter', 'y', 'x']
extra_coord_vars = []
chunks = dict(time_counter=1)
open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),
                   chunks=chunks,
                   decode_cf=True,
                   decode_times=False,
                   concat_dim="time_counter")  #, combine='nested')


In [6]:
ds = xr.open_mfdataset(files, parallel=True, **open_kwargs)


In [7]:
template = xr.open_dataset(files[0], decode_cf=False, decode_times=False)

ds["nav_lat"] = template["nav_lat"]

ds["nav_lon"] = template["nav_lon"]

ds

del template


In [8]:
ds

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, time_counter: 9528, x: 8354, y: 4729)
Coordinates:
    time_centered         (time_counter) float64 dask.array<shape=(9528,), chunksize=(1,)>
  * time_counter          (time_counter) float64 3.455e+09 ... 3.49e+09
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    time_centered_bounds  (time_counter, axis_nbounds) float64 dask.array<shape=(9528, 2), chunksize=(1, 2)>
    time_counter_bounds   (time_counter, axis_nbounds) float64 dask.array<shape=(9528, 2), chunksize=(1, 2)>
    sossheig              (time_counter, y, x) float32 dask.array<shape=(9528, 4729, 8354), chunksize=(1, 4729, 8354)>
    nav_lat               (y, x) float32 ...
    nav_lon               (y, x) float32 ...
Attributes:
    name:         /scratch/tmp/3735876/eNATL60-BLB002_1h_20090630_20090704_gr...
    description:  ocean T grid variables
    title:        ocean T grid variables
    Conventions:  CF-1.6
    timeStamp:    2018-Dec-30 1

In [9]:
compressor = numcodecs.Blosc(cname='snappy', clevel=6, shuffle=-1)
encoding = {vname: {'compressor': compressor} for vname in ds.variables}


In [13]:
cluster.scale(360)
cluster

from dask.distributed import Client
client = Client(cluster)
client


Client Scheduler: tcp://172.30.100.1:46174 Dashboard: http://172.30.100.1:44972/status,Cluster Workers: 280 Cores: 280 Memory: 1.20 TB


In [14]:
outdir = '/store/albert7a/eNATL60/zarr/eNATL60-BLB002X-SSH-1h-ter'
ds = ds.chunk(chunks=dict(time_counter=240, y=240, x=480))
ds.to_zarr(outdir, encoding=encoding, mode="w")

 

distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

KilledWorker: ("('open_dataset-concatenate-a1b7a7e4e78b73a625455db73dfa122b', 6703, 0)", <Worker 'tcp://172.30.5.134:34257', memory: 0, processing: 178>)

distributed.core - ERROR - 'tcp://172.30.6.100:46143'
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/core.py", line 471, in handle_stream
    handler(**merge(extra, msg))
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py", line 2326, in handle_release_data
    ws = self.workers[worker]
KeyError: 'tcp://172.30.6.100:46143'
distributed.utils - ERROR - 'tcp://172.30.6.100:46143'
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py", line 1500, in add_worker
    yield self.handle_worker(comm=comm, worker=address)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 735, in run
    value = 